In [1]:
# Importacion de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

%matplotlib inline

plt.style.use('default') # para graficos matplotlib
plt.rcParams['figure.figsize'] = (10, 8)

sns.set(style="whitegrid") # grid seaborn

# pd.options.display.float_format = '{:20,.0f}'.format # notacion output

# Acceso a set de datos

In [2]:
path = "data/"
df_props_full = pd.read_csv(path + "train_small_provinces.csv")
df_test = pd.read_csv(path + "test_small_provinces.csv")

In [3]:
df_props_full.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [4]:
df_test.columns

Index(['Unnamed: 0', 'id', 'titulo', 'descripcion', 'tipodepropiedad',
       'direccion', 'ciudad', 'provincia', 'antiguedad', 'habitaciones',
       'garages', 'banos', 'metroscubiertos', 'metrostotales', 'idzona', 'lat',
       'lng', 'fecha', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos'],
      dtype='object')

In [5]:
df_test.id.sort_values()

671       369
393       931
641      1226
141      1253
127      1443
        ...  
205    297741
581    297787
162    298089
541    299629
194    299644
Name: id, Length: 737, dtype: int64

In [6]:
df_props_full.id.sort_values()

2807        36
2424        78
2553       207
1405       214
668        264
         ...  
1009    299222
2679    299385
1699    299489
2076    299584
2762    299647
Name: id, Length: 2901, dtype: int64

# Conversión de tipos

In [7]:
df_props_full['fecha'] = pd.to_datetime(df_props_full['fecha'])
df_test['fecha'] = pd.to_datetime(df_test['fecha'])

In [8]:
# Convierto todos los valores 1/0 a uint8
df_props_full['gimnasio'] = df_props_full['gimnasio'].astype('uint8')
df_props_full['usosmultiples'] = df_props_full['usosmultiples'].astype('uint8')
df_props_full['piscina'] = df_props_full['piscina'].astype('uint8')
df_props_full['escuelascercanas'] = df_props_full['escuelascercanas'].astype('uint8')
df_props_full['centroscomercialescercanos'] = df_props_full['centroscomercialescercanos'].astype('uint8')

df_test['gimnasio'] = df_test['gimnasio'].astype('uint8')
df_test['usosmultiples'] = df_test['usosmultiples'].astype('uint8')
df_test['piscina'] = df_test['piscina'].astype('uint8')
df_test['escuelascercanas'] = df_test['escuelascercanas'].astype('uint8')
df_test['centroscomercialescercanos'] = df_test['centroscomercialescercanos'].astype('uint8')

In [9]:
# Convierto los representables en uint8. Utilizo el tipo de pandas UInt8Dtype para evitar conflicto con NaN
df_props_full['antiguedad'] = df_props_full['antiguedad'].astype(pd.UInt8Dtype())
df_props_full['habitaciones'] = df_props_full['habitaciones'].astype(pd.UInt8Dtype())
df_props_full['garages'] = df_props_full['garages'].astype(pd.UInt8Dtype())
df_props_full['banos'] = df_props_full['banos'].astype(pd.UInt8Dtype())

df_test['antiguedad'] = df_test['antiguedad'].astype(pd.UInt8Dtype())
df_test['habitaciones'] = df_test['habitaciones'].astype(pd.UInt8Dtype())
df_test['garages'] = df_test['garages'].astype(pd.UInt8Dtype())
df_test['banos'] = df_test['banos'].astype(pd.UInt8Dtype())

In [10]:
# Convierto los representables en uint16. Utilizo el tipo de pandas UInt16Dtype para evitar conflicto con NaN
df_props_full['metroscubiertos'] = df_props_full['metroscubiertos'].astype(pd.UInt16Dtype())
df_props_full['metrostotales'] = df_props_full['metrostotales'].astype(pd.UInt16Dtype())

df_test['metroscubiertos'] = df_test['metroscubiertos'].astype(pd.UInt16Dtype())
df_test['metrostotales'] = df_test['metrostotales'].astype(pd.UInt16Dtype())

In [11]:
# Convierto los representables en uint32. Utilizo el tipo de pandas UInt32Dtype para evitar conflicto con NaN
df_props_full['id'] = df_props_full['id'].astype(pd.UInt32Dtype())
df_props_full['idzona'] = df_props_full['idzona'].astype(pd.UInt32Dtype())
# df_props_full['Precio_MEX'] = df_props_full['Precio_MEX'].astype(pd.UInt32Dtype())
# df_props_full['Precio_USD'] = df_props_full['Precio_USD'].astype(pd.UInt32Dtype())
df_props_full['precio'] = df_props_full['precio'].astype('float32')

df_test['id'] = df_test['id'].astype(pd.UInt32Dtype())
df_test['idzona'] = df_test['idzona'].astype(pd.UInt32Dtype())

# Habitaciones

Basado en información obtenida del TP1 elimino habitaciones con valores nulos

In [12]:
df_props_full.habitaciones.isna().value_counts()

False    2503
True      398
Name: habitaciones, dtype: int64

In [13]:
df_props_full.habitaciones.value_counts()

3     1374
2      691
4      285
5       67
1       55
6        9
8        6
10       6
7        6
9        4
Name: habitaciones, dtype: int64

In [14]:
features = df_props_full
len(features)

2901

In [15]:
pd.pivot_table(df_props_full.groupby(['tipodepropiedad','habitaciones']).size().to_frame(), index='tipodepropiedad',columns=['habitaciones'])

0                                        \
habitaciones                     1      2       3      4     5    6    7    
tipodepropiedad                                                             
Apartamento                     2.0  115.0    58.0    3.0   NaN  NaN  NaN   
Casa                           43.0  533.0  1263.0  268.0  65.0  9.0  5.0   
Casa en condominio              3.0   33.0    44.0    8.0   NaN  NaN  NaN   
Casa uso de suelo               NaN    1.0     1.0    1.0   1.0  NaN  NaN   
Edificio                        1.0    NaN     NaN    NaN   NaN  NaN  NaN   
Hospedaje                       NaN    NaN     NaN    NaN   NaN  NaN  1.0   
Inmuebles productivos urbanos   NaN    NaN     NaN    NaN   NaN  NaN  NaN   
Local Comercial                 1.0    2.0     NaN    NaN   1.0  NaN  NaN   
Oficina comercial               NaN    NaN     NaN    1.0   NaN  NaN  NaN   
Otros                           1.0    NaN     1.0    NaN   NaN  NaN  NaN   
Quinta Vacacional               2.0    1.0     6.0    2.0   NaN  NaN  NaN   
Rancho                          NaN    1.0     NaN    NaN   NaN  NaN  NaN   
Terreno                         2.0    3.0     1.0    1.0   NaN  NaN  NaN   
Terreno comercial               NaN    1.0     NaN    1.0   NaN  NaN  NaN   
Villa                           NaN    1.0     NaN    NaN   NaN  NaN  NaN   

                                              
habitaciones                    8    9    10  
tipodepropiedad                               
Apartamento                    NaN  NaN  NaN  
Casa                           4.0  4.0  4.0  
Casa en condominio             NaN  NaN  NaN  
Casa uso de suelo              1.0  NaN  NaN  
Edificio                       1.0  NaN  1.0  
Hospedaje                      NaN  NaN  NaN  
Inmuebles productivos urbanos  NaN  NaN  1.0  
Local Comercial                NaN  NaN  NaN  
Oficina comercial              NaN  NaN  NaN  
Otros                          NaN  NaN  NaN  
Quinta Vacacional              NaN  NaN  NaN  
Rancho                         NaN  NaN  NaN  
Terreno                        NaN  NaN  NaN  
Terreno comercial              NaN  NaN  NaN  
Villa                          NaN  NaN  NaN

In [16]:
features = features.dropna(subset=['lat', 'lng', 'habitaciones'])

print(type(features))
len(features)

<class 'pandas.core.frame.DataFrame'>


1325

# Baños

Basado en información obtenida del TP1 elimino baños con valores nulos

In [17]:
df_props_full.banos.isna().value_counts()

False    2442
True      459
Name: banos, dtype: int64

In [18]:
features.banos.isna().value_counts()

False    1276
True       49
Name: banos, dtype: int64

In [19]:
features.banos.value_counts()

1    490
2    489
3    223
4     74
Name: banos, dtype: int64

In [20]:
pd.pivot_table(df_props_full.groupby(['tipodepropiedad','banos']).size().to_frame(), index='tipodepropiedad',columns=['banos'])

0                     
banos                   1      2      3      4
tipodepropiedad                               
Apartamento         107.0   51.0   13.0    5.0
Bodega comercial      6.0    3.0    1.0    1.0
Casa                767.0  798.0  417.0  128.0
Casa en condominio   31.0   28.0   19.0    6.0
Casa uso de suelo     1.0    1.0    1.0    NaN
Edificio              3.0    2.0    1.0    2.0
Local Comercial      10.0    6.0    3.0    2.0
Oficina comercial     2.0    1.0    1.0    1.0
Otros                 2.0    1.0    NaN    NaN
Quinta Vacacional     2.0    3.0    4.0    1.0
Rancho                1.0    NaN    NaN    NaN
Terreno               6.0    1.0    NaN    NaN
Terreno comercial     NaN    1.0    1.0    NaN
Villa                 1.0    NaN    NaN    NaN

In [21]:
features = features.dropna(subset=['banos'])

print(type(features))
len(features)

<class 'pandas.core.frame.DataFrame'>


1276

# Garages

In [22]:
df_props_full.garages.isna().value_counts()

False    2417
True      484
Name: garages, dtype: int64

In [23]:
features.garages.isna().value_counts()

False    1084
True      192
Name: garages, dtype: int64

In [24]:
features.garages.value_counts()

1    468
2    461
0    101
3     54
Name: garages, dtype: int64

In [25]:
pd.pivot_table(df_props_full.groupby(['tipodepropiedad','garages']).size().to_frame(), index='tipodepropiedad',columns=['garages'])

0                     
garages                            0      1      2      3
tipodepropiedad                                          
Apartamento                     19.0  110.0   30.0    3.0
Bodega comercial                 3.0    3.0    NaN    1.0
Casa                           207.0  799.0  822.0  103.0
Casa en condominio               4.0   26.0   31.0    5.0
Casa uso de suelo                1.0    2.0    1.0    1.0
Edificio                        12.0    1.0    2.0    1.0
Hospedaje                        1.0    NaN    NaN    NaN
Huerta                           1.0    NaN    NaN    NaN
Inmuebles productivos urbanos    1.0    NaN    NaN    NaN
Local Comercial                 17.0    2.0    1.0    NaN
Local en centro comercial        1.0    NaN    NaN    NaN
Oficina comercial                5.0    2.0    1.0    1.0
Quinta Vacacional                1.0    NaN    2.0    4.0
Rancho                           1.0    NaN    NaN    NaN
Terreno                        172.0    1.0    1.0    3.0
Terreno comercial               12.0    NaN    NaN    NaN

In [26]:
pd.pivot_table(df_props_full.groupby(['tipodepropiedad','garages']).size().to_frame(), index='tipodepropiedad',columns=['garages'])

0                     
garages                            0      1      2      3
tipodepropiedad                                          
Apartamento                     19.0  110.0   30.0    3.0
Bodega comercial                 3.0    3.0    NaN    1.0
Casa                           207.0  799.0  822.0  103.0
Casa en condominio               4.0   26.0   31.0    5.0
Casa uso de suelo                1.0    2.0    1.0    1.0
Edificio                        12.0    1.0    2.0    1.0
Hospedaje                        1.0    NaN    NaN    NaN
Huerta                           1.0    NaN    NaN    NaN
Inmuebles productivos urbanos    1.0    NaN    NaN    NaN
Local Comercial                 17.0    2.0    1.0    NaN
Local en centro comercial        1.0    NaN    NaN    NaN
Oficina comercial                5.0    2.0    1.0    1.0
Quinta Vacacional                1.0    NaN    2.0    4.0
Rancho                           1.0    NaN    NaN    NaN
Terreno                        172.0    1.0    1.0    3.0
Terreno comercial               12.0    NaN    NaN    NaN

Basado en informacion del TP1 completo con 0 aquellas propiedades que muestren una tendencia a este valor y elimino el resto

In [27]:
aCompletar = ['Edificio','Bodega comercial','Terreno comercial','Local comercial','Terreno','Rancho','Inmuebles productivos urbanos','Nave industrial','Local en centro comercial']
features.loc[(features.tipodepropiedad.isin(aCompletar)), 'garages'] = features.loc[(features.tipodepropiedad.isin(aCompletar)), 'garages'].fillna(0)
df_test.loc[(df_test.tipodepropiedad.isin(aCompletar)), 'garages'] = df_test.loc[(df_test.tipodepropiedad.isin(aCompletar)), 'garages'].fillna(0)


len(features)

1276

In [28]:
features.loc[(features['tipodepropiedad']=='Casa'),:].garages.isna().value_counts()

False    962
True     166
Name: garages, dtype: int64

In [29]:
features.loc[(features['tipodepropiedad']=='Casa en condominio'),:].garages.isna().value_counts()

False    31
True      7
Name: garages, dtype: int64

In [30]:
features.loc[(features['tipodepropiedad']=='Villa'),:].garages.isna().value_counts()

True    1
Name: garages, dtype: int64

Completo estas tres propiedades con valor 2 que tiene mayor cantidad de publicaciones

In [31]:
aCompletar2 = ['Casa', 'Casa en condominio', 'Villa']
features.loc[(features.tipodepropiedad.isin(aCompletar2)), 'garages'] = features.loc[(features.tipodepropiedad.isin(aCompletar2)), 'garages'].fillna(2)
df_test.loc[(df_test.tipodepropiedad.isin(aCompletar2)), 'garages'] = df_test.loc[(df_test.tipodepropiedad.isin(aCompletar2)), 'garages'].fillna(2)

len(features)

1276

In [32]:
features.garages.isna().value_counts()

False    1259
True       17
Name: garages, dtype: int64

In [33]:
features = features.dropna(subset=['garages'])

len(features)

1259

# Metros

In [34]:
from sklearn.impute import SimpleImputer

In [35]:
imp = SimpleImputer()
features['metroscubiertos'] = imp.fit_transform(features[['metroscubiertos']])
features['metrostotales'] = imp.fit_transform(features[['metrostotales']])

df_test['metroscubiertos'] = imp.fit_transform(df_test[['metroscubiertos']])
df_test['metrostotales'] = imp.fit_transform(df_test[['metrostotales']])

# Antiguedad

In [36]:
features['antiguedad'] = imp.fit_transform(features[['antiguedad']])

df_test['antiguedad'] = imp.fit_transform(df_test[['antiguedad']])

# Tipo de propiedad

In [37]:
features.tipodepropiedad.isna().value_counts()

False    1259
Name: tipodepropiedad, dtype: int64

In [38]:
# Drop de datos sin tipo
features = features.dropna(subset=['tipodepropiedad'])
len(features)

1259

# Ciudad

In [39]:
def ciudadesByProvinciaMap(df):
    provs_ciudades = {}
    for i in range(0,len(df)):
        prov = df.loc[i,'provincia']
        city = df.loc[i,'ciudad']
        if ((not pd.isna(prov)) and (not pd.isna(city))):
            if (not prov in provs_ciudades):
                provs_ciudades[prov] = []
            if (not city in provs_ciudades[prov]):
                provs_ciudades[prov].append(city)
    return provs_ciudades

In [40]:
def provinciaByIdZonaMap(df):
    id_prov = {}
    for i in range(0,len(df)):
        prov = df.loc[i,'provincia']
        idzona = df.loc[i,'idzona']
        if ((not pd.isna(prov)) and (not pd.isna(idzona))):
            if (not idzona in id_prov):
                id_prov[idzona] = []
            if (not prov in id_prov[idzona]):
                id_prov[idzona].append(prov)
    return id_prov

# Provincia

One hot encoding

In [41]:
one_hot = pd.get_dummies(features.provincia)
features = features.drop('provincia',axis = 1)
features = features.join(one_hot)

one_hot_test = pd.get_dummies(df_test.provincia)
df_test = df_test.drop('provincia',axis = 1)
df_test = df_test.join(one_hot_test)

features = features.reset_index(drop=True)  # Corrijo index con rows eliminadas para que funcione el for

# Fix Test data set

In [42]:
len(df_test)

737

In [43]:
cols_dropped = ['fecha', 'id', 'idzona']
cols_string = ['titulo', 'descripcion','tipodepropiedad','direccion', 'ciudad']
df_test.drop(cols_dropped + cols_string, axis=1).count()

Unnamed: 0                    737
antiguedad                    737
habitaciones                  648
garages                       717
banos                         631
metroscubiertos               737
metrostotales                 737
lat                           406
lng                           406
gimnasio                      737
usosmultiples                 737
piscina                       737
escuelascercanas              737
centroscomercialescercanos    737
Campeche                      737
Oaxaca                        737
Tabasco                       737
Tlaxcala                      737
Zacatecas                     737
dtype: int64

In [44]:
df_test['habitaciones'] = imp.fit_transform(df_test[['habitaciones']])
df_test['garages'] = imp.fit_transform(df_test[['garages']])
df_test['banos'] = imp.fit_transform(df_test[['banos']])

In [45]:
cols_dropped = ['fecha', 'idzona']
cols_string = ['titulo', 'descripcion','tipodepropiedad','direccion', 'ciudad']
df_test.drop(cols_dropped + cols_string, axis=1).count()

Unnamed: 0                    737
id                            737
antiguedad                    737
habitaciones                  737
garages                       737
banos                         737
metroscubiertos               737
metrostotales                 737
lat                           406
lng                           406
gimnasio                      737
usosmultiples                 737
piscina                       737
escuelascercanas              737
centroscomercialescercanos    737
Campeche                      737
Oaxaca                        737
Tabasco                       737
Tlaxcala                      737
Zacatecas                     737
dtype: int64

# Features a CSV

In [46]:
cols_dropped = ['lat', 'lng','fecha', 'id', 'idzona']
cols_string = ['titulo', 'descripcion','tipodepropiedad','direccion', 'ciudad']
features = features.drop(cols_dropped + cols_string, axis=1)
cols_dropped = ['lat', 'lng', 'fecha', 'idzona']
df_test = df_test.drop(cols_dropped + cols_string, axis=1)

In [47]:
features.to_csv('data/features_small_provinces.csv', index=False)
df_test.to_csv('data/test_small_provinces_modified.csv', index=True)